# Getting Started with Structured Outputs in Mirascope

This notebook provides a detailed introduction to implementing evaluations using Mirascope, focusing on using LLMs as judges and implementing hardcoded evaluation criteria.

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Extracting Structured Outputs](#Extracting-Structured-Outputs)
4. [Generating Structured Outputs](#Generating-Structured-Outputs)
5. [JSON Mode](#JSON-Mode)
6. [Few-Shot Examples](#Few-Shot-Examples)
7. [Validating Outputs](#Validating-Outputs)
8. [Output Parsers](#Output-Parsers)

## Introduction

Large Language Models (LLMs) generate unstructured text data by default. Structured outputs are essential for building reliable and efficient AI applications, and this notebook demonstrates various techniques for structuring LLM outputs using Mirascope.

These methods help ensure consistency, type safety, and easier integration of LLM responses into your application. For more detailed information on structured outputs in Mirascope, refer to the [Response Models](https://docs.mirascope.io/latest/learn/response_models) documentation, [JSON Mode](https://docs.mirascope.io/latest/learn/json_mode) documentation, and [Output Parser](https://docs.mirascope.io/latest/learn/output_parsers) documentation.

## Setup

First, let's set up our environment by installing Mirascope and importing the necessary modules.

In [1]:
!pip install "mirascope[openai]"

In [ ]:
import os

# Set your API keys
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

## Extracting Structured Outputs

The simplest way to extract structured outputs with Mirascope is using `response_model` to define the output type of the call:

In [2]:
from mirascope.core import openai, prompt_template
from pydantic import BaseModel


class Book(BaseModel):
    title: str
    author: str


@openai.call("gpt-4o-mini", response_model=Book)
@prompt_template()
def extract_book(text: str) -> str:
    return f"Extract the book from this text: {text}"


book = extract_book("The Name of the Wind by Patrick Rothfuss")
assert isinstance(book, Book)
print(book)

title='The Name of the Wind' author='Patrick Rothfuss'


In this example we are:
1. Defining a response model `Book` as a Pydantic `BaseModel` subclass
2. Setting `response_model` equal to our `Book` type.
3. Running our LLM API call function as normal, except the output is now a `Book` instance.

## Generating Structured Outputs

Another common use-case for structured outputs is to generate synthetic data. The interface is the same, requiring only an update to the prompt:

In [3]:
@openai.call("gpt-4o-mini", response_model=list[Book])
@prompt_template()
def recommend_books(genre: str, num: int) -> str:
    return f"Recommend a list of {num} {genre} books"


books = recommend_books("fantasy", 3)
for book in books:
    print(book)

title='The Name of the Wind' author='Patrick Rothfuss'
title='Mistborn: The Final Empire' author='Brandon Sanderson'
title='A Darker Shade of Magic' author='V.E. Schwab'


In this example we:
1. Updated our prompt to instruct to model to "recommend" (i.e. generate) books.
2. We set `response_model` equal to `list[Book]` to output multiple books instead of just one.
3. We further updated our prompt to enable the user to specify how many books to generate.

## JSON Mode

Many LLM providers have JSON Mode to instruct the model to output JSON. Although not all providers offer JSON Mode support officially, Mirascope offers support for all providers. For providers with official support, we simply use the native API feature. For providers without official support, we prompt engineer the model to give us the JSON:

In [11]:
import json


@openai.call("gpt-4o-mini", json_mode=True)
@prompt_template()
def extract_book(text: str) -> str:
    return f"Extract the book title and author from this text: {text}"


response = extract_book("The Name of the Wind by Patrick Rothfuss")
print(json.loads(response.content))

{'title': 'The Name of the Wind', 'author': 'Patrick Rothfuss'}


In this example we:
1. Set `json_mode=True` to signal we want to use JSON Mode
2. Specify the fields that we want in the prompt
3. Parse the json string output into a Python dictionary

If you want additional validation on the output structure and types, you can use `json_mode` in conjunction with `response_model` to validate your structured outputs:



In [12]:
@openai.call("gpt-4o-mini", response_model=Book, json_mode=True)
@prompt_template()
def extract_book(text: str) -> str:
    return f"Extract the book from this text: {text}"


book = extract_book("The Name of the Wind by Patrick Rothfuss")
assert isinstance(book, Book)
print(book)

title='The Name of the Wind' author='Patrick Rothfuss'


## Few-Shot Examples

Often when guiding an LLM's response, providing few-shot examples can greatly help steer the output in the right direction:

In [19]:
from pydantic import Field, ConfigDict


class FewShotBook(BaseModel):
    title: str = Field(..., examples=["THE NAME OF THE WIND"])
    author: str = Field(..., examples=["Rothfuss, Patrick"])

    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {"title": "THE NAME OF THE WIND", "author": "Rothfuss, Patrick"},
            ]
        }
    )


@openai.call("gpt-4o-mini", response_model=list[FewShotBook], json_mode=True)
@prompt_template()
def recommend_few_shot_books(genre: str, num: int) -> str:
    return f"Recommend a list of {num} {genre} books. Match example format."


books = recommend_few_shot_books("fantasy", 3)
for book in books:
    print(book)

title='THE LORD OF THE RINGS' author='Tolkien, J.R.R.'
title='MISTBORN: THE FINAL EMPIRE' author='Sanderson, Brandon'
title='GARDENS OF THE MOON' author='Erikson, Steven'


In this example we:
1. Added a few-shot example to each field in our response model.
2. Added a few-shot example for the entire response model.
3. Set `json_mode=True` because we have found that examples are more effective with this setting.
4. Updated the prompt to instruct the LLM to match the format of the examples.

## Validating Outputs

Since `response_model` relies on Pydantic `BaseModel` types, you can easily add additional validation criteria to your model. This ensures the validation will run on every call:

In [24]:
from pydantic import field_validator


class ValidatedBook(BaseModel):
    title: str
    author: str

    @field_validator("title", "author")
    @classmethod
    def must_be_uppercase(cls, v: str) -> str:
        assert v.isupper(), "All fields must be uppercase"
        return v


@openai.call("gpt-4o-mini", response_model=ValidatedBook)
@prompt_template()
def extract_all_caps_book(text: str) -> str:
    return f"Extract the book from this text: {text}"


book = extract_all_caps_book("The Name of the Wind by Patrick Rothfuss")
print(book)

ValidationError: 2 validation errors for ValidatedBook
title
  Assertion failed, All fields must be uppercase [type=assertion_error, input_value='The Name of the Wind', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/assertion_error
author
  Assertion failed, All fields must be uppercase [type=assertion_error, input_value='Patrick Rothfuss', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/assertion_error

We can see in this example that the model threw a `ValidationError` on construction because the extracted `title` and `author` fields were not all caps.

This example is of course for demonstration purposes. In a real-world example we would ensure we catch such errors and handle them gracefully as well as further engineer the prompt to ensure such errors occur rarely or not at all.

## Output Parsers

Mirascope also provides an `output_parser` argument that will run on the call response for every call. This enables writing prompts that have a more specific structure (such as Chain of Thought) while still ensuring the output has the desired structure:

In [23]:
import re


def parse_cot(response: openai.OpenAICallResponse) -> tuple[str, str]:
    pattern = r"<thinking>(.*?)</thinking>.*?<output>(.*?)</output>"
    match = re.search(pattern, response.content, re.DOTALL)
    if not match:
        return "", response.content
    else:
        return match.group(1).strip(), match.group(2).strip()


@openai.call("gpt-4o-mini", output_parser=parse_cot)
@prompt_template(
    """
    SYSTEM:
    First, output your thought process in <thinking> tags.
    Then, provide your final output in <output> tags.

    USER: {query}
    """
)
def cot(query: str): ...


output = cot(
    "How many tennis balls does Roger have if he started with 2 and bought 3 tubes?"
)
print(f"Thinking: {output[0]}")
print(f"Output: {output[1]}")

Thinking: To solve the problem, I need to understand how many tennis balls are in a tube. Typically, a standard tube of tennis balls contains 3 balls. Since Roger starts with 2 tennis balls and then buys 3 tubes, I can calculate how many balls he gets from the tubes and then add that to the 2 he already has. First, I'll calculate the total number of tennis balls from the 3 tubes, which is 3 balls per tube multiplied by 3 tubes. Then, I’ll add the 2 balls he started with.
Output: Roger has a total of 11 tennis balls. (2 starting balls + 9 from 3 tubes)


## Conclusion

These techniques provide a solid foundation for structuring outputs in your LLM applications. As you continue to work with LLMs and develop more complex applications, robust prompt engineering and validation will be crucial for ensuring the quality and reliability of your models and outputs.

For more advanced topics and best practices, refer to the Mirascope [Response Models](https://docs.mirascope.io/latest/learn/response_models) documentation, [JSON Mode](https://docs.mirascope.io/latest/learn/json_mode) documentation, and [Output Parsers](https://docs.mirascope.io/latest/learn/output_parsers) documentation.

We also recommend taking a look at our [Tenacity Integration](https://docs.mirascope.io/latest/integrations/tenacity/) to learn how to build more robust pipelines that automatically re-insert validation errors into a subsequent call, enabling the LLM to learn from its mistakes and (hopefully) output the correct answer on the next attempt.